In [ ]:
## Libraries Required
!pip install langchain-huggingface
## For API Calls
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
import json
from langchain import PromptTemplate, LLMChain
from langchain_huggingface import HuggingFaceEndpoint

# Load the Mistral-7B-Instruct-v0.2 model from Hugging Face API
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=HUGGINGFACE_TOKEN,
    temperature=0.7,
    max_new_tokens=1024  # Increased to allow full JSON output
)

# JSON response format
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

TEMPLATE = """
Text: {text}
You are an expert MCQ generator. Based on the text above, create {number} multiple-choice questions 
for {subject} students at a difficulty level of {tone}. 

Your response **MUST be in valid JSON format**. Follow these strict rules:
1. **DO NOT** include any explanations, markdown, or extra text—**return ONLY valid JSON**.
2. **Ensure JSON syntax is correct**, with proper quotes and formatting.
3. **Use unique, non-repetitive questions** strictly based on the text.
4. **Options should be diverse and not misleading**.
5. **Each question should have exactly four options (A, B, C, D)** and one correct answer.

### **JSON RESPONSE FORMAT**
```json
{response_json}"""


# Example input
TEXT = """Electromagnetic induction is the process of generating an electromotive force (EMF) or voltage across
a conductor when exposed to a changing magnetic field. This principle was discovered by Michael Faraday in 1831
and is the basis for electrical generators and transformers.

According to Faraday’s Law, the magnitude of the induced EMF is directly proportional to the rate of change
of magnetic flux through the conductor. Mathematically, it is expressed as:

𝓔 = -dΦB/dt

where 𝓔 is the induced EMF, and ΦB is the magnetic flux. The negative sign represents Lenz’s Law, which states
that the induced current will always oppose the change in magnetic flux that caused it.

There are two main types of electromagnetic induction: mutual induction and self-induction. Mutual induction
occurs when a changing current in one coil induces an EMF in a nearby coil, as seen in transformers. Self-induction
occurs when a changing current in a coil induces an EMF in the same coil due to its own changing magnetic field.

This principle has widespread applications, including power generation, wireless charging, and credit card readers."""

# Create a LangChain prompt template
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

# Define the LangChain LLMChain
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=False)

# Parameters
NUMBER = 5
SUBJECT = "Electromagnetic Induction"
TONE = "easy"

# Generate MCQs
response = quiz_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON, indent=4)
})

# Extract the JSON part from response and parse it
quiz_json_str = response["quiz"].strip()

In [ ]:
import json
import re

# Original text containing the JSON
text = quiz_json_str

# Regular expression to extract JSON content
json_string = re.search(r'```json(.*?)```', text, re.DOTALL).group(1).strip()

# Convert the string to a JSON object
data = json.loads(json_string)

# Output the JSON data
print(json.dumps(data, indent=4))
